
* this [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/159_prototype.ipynb)
 

---- 
* Issue [159 gör en prototyp (GeoJSON/JSON-LD + API-exempel) som visar hur data om SAT-faciliteter kan delas öppet](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/159)
  * also added a version 2 
 

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-09-10 12:19:50


In [2]:
# sat_map.ipynb
import requests
import folium
import geopandas as gpd
from shapely.geometry import Point
from SPARQLWrapper import SPARQLWrapper, JSON

# --- 1. Hämta SAT-punkter från Wikidata ---
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel ?coord ?image WHERE {
  ?item wdt:P6104 wd:Q134294510;  # SAT
        wdt:P625 ?coord.
  OPTIONAL { ?item wdt:P18 ?image. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

wd_points = []
for r in results["results"]["bindings"]:
    coord = r["coord"]["value"].replace("Point(", "").replace(")", "").split(" ")
    lon, lat = map(float, coord)
    wd_points.append({
        "name": r["itemLabel"]["value"],
        "lat": lat,
        "lon": lon,
        "image": r.get("image", {}).get("value")
    })

gdf_wd = gpd.GeoDataFrame(
    wd_points, geometry=[Point(p["lon"], p["lat"]) for p in wd_points], crs="EPSG:4326"
)

# --- 2. Hämta OSM-data längs leden (toalett + dricksvatten) ---
# Exempel: Overpass API query
overpass_url = "https://overpass-api.de/api/interpreter"
query = """
[out:json][timeout:50];
(
  node["amenity"="toilets"](59.0,18.0,60.5,19.5);
  node["amenity"="drinking_water"](59.0,18.0,60.5,19.5);
);
out center;
"""
osm_data = requests.get(overpass_url, params={'data': query}).json()

osm_points = []
for e in osm_data["elements"]:
    osm_points.append({
        "name": e["tags"].get("amenity"),
        "lat": e["lat"],
        "lon": e["lon"],
    })

gdf_osm = gpd.GeoDataFrame(
    osm_points, geometry=[Point(p["lon"], p["lat"]) for p in osm_points], crs="EPSG:4326"
)

# --- 3. Exportera GeoJSON ---
gdf_wd.to_file("output/159_sat_points.geojson", driver="GeoJSON")
gdf_osm.to_file("output/159_sat_facilities.geojson", driver="GeoJSON")

# --- 4. Skapa interaktiv karta ---
m = folium.Map(location=[59.5, 18.8], zoom_start=8)

for _, row in gdf_wd.iterrows():
    folium.Marker(
        [row["lat"], row["lon"]],
        popup=row["name"],
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(m)

for _, row in gdf_osm.iterrows():
    folium.Marker(
        [row["lat"], row["lon"]],
        popup=row["name"],
        icon=folium.Icon(color="blue", icon="tint")
    ).add_to(m)

m.save("output/159_sat_map.html")
print("✅ Export klar: 159_sat_points.geojson, 159_sat_facilities.geojson, 159_sat_map.html")


✅ Export klar: 159_sat_points.geojson, 159_sat_facilities.geojson, 159_sat_map.html


In [3]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))


Date: 2025-09-10 12:19:54
Total time elapsed: 4.12 seconds


## Version 2.3

In [ ]:
# sat_map_v2.ipynb — FULL SCRIPT (v2.3)
# Changes in v2.3:
# - Robust Overpass handling: multiple mirrors, POST, retries with exponential backoff
# - Per-section queries simplified: `relation(..); way(r); out tags geom;` or `way(..); out tags geom;`
# - Fixed `parts_str` newline join bug
# - Keeps wheelchair/paper layers + legend

import os
import re
import html
import json
import time
import requests
import folium
import geopandas as gpd
from shapely.geometry import Point, LineString
from SPARQLWrapper import SPARQLWrapper, JSON
from folium.plugins import MarkerCluster

# -------------------- CONFIG --------------------
CONFIG = {
    "trail_qid": "Q131318799",            # Stockholm Archipelago Trail
    "map_center": [59.5, 18.8],            # Initial map view
    "bbox": (59.0, 18.0, 60.5, 19.5),      # (south, west, north, east)
    # Outputs (versioned)
    "out_dir": "output",
    "out_points": "output/159_2_sat_points.geojson",
    "out_facilities": "output/159_2_sat_facilities.geojson",
    "out_sections": "output/159_2_sat_sections.geojson",
    "out_html": "output/159_2_sat_map.html",
    # UI
    "thumb_width": 500,
    "trail_color": "#8B0000",  # dark red
    "trail_weight": 4,
}

# -------------------- HELPERS --------------------

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)


def extract_commons_filename(url: str) -> str | None:
    if not url:
        return None
    m = re.search(r"/FilePath/([^?#]+)", url)
    if m:
        return m.group(1)
    m2 = re.search(r"/(?:wiki|w)/(?:File:|file:)([^?#]+)", url)
    if m2:
        return m2.group(1)
    return None


def commons_thumb_url(p18_url: str, width: int = 400) -> str | None:
    fn = extract_commons_filename(p18_url)
    if not fn:
        return None
    return f"https://commons.wikimedia.org/wiki/Special:FilePath/{fn}?width={width}"

# Overpass mirrors and robust request
OVERPASS_URLS = [
    "https://overpass-api.de/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter",
    "https://overpass.nchc.org.tw/api/interpreter",
    "https://overpass.openstreetmap.ru/api/interpreter",
]
USER_AGENT = {"User-Agent": "SAT-map-v2/1.0 (salgo60 prototype)"}

def overpass(query: str, retries: int = 3, backoff: float = 2.0) -> dict:
    last_err = None
    for attempt in range(retries):
        for base in OVERPASS_URLS:
            try:
                r = requests.post(base, data={"data": query}, headers=USER_AGENT, timeout=180)
                # Retry on common transient codes
                if r.status_code in (429, 502, 503, 504):
                    last_err = requests.HTTPError(f"{r.status_code} from {base}")
                    continue
                r.raise_for_status()
                return r.json()
            except Exception as e:
                last_err = e
                continue
        time.sleep(backoff * (attempt + 1))
    if last_err:
        raise last_err
    return {"elements": []}


def sss_like_url(url: str | None) -> bool:
    if not url:
        return False
    return any(host in url for host in ["skargardsstiftelsen.se", "skärgårdsstiftelsen.se"])  # heuristic

# Helper: try find an OSM `image=*` near a coordinate (fallback when P18 missing)
def find_osm_image_near(lat: float, lon: float, radius: int = 40) -> str | None:
    q = f"""
    [out:json][timeout:40];
    (
      node(around:{radius},{lat},{lon})["image"];
      way(around:{radius},{lat},{lon})["image"];
      relation(around:{radius},{lat},{lon})["image"];
    );
    out tags center 1;
    """
    try:
        data = overpass(q, retries=2)
        for el in data.get("elements", [])[:3]:
            img = (el.get("tags") or {}).get("image")
            if isinstance(img, str) and img.strip():
                return img.strip()
    except Exception:
        pass
    return None

def thumb_from_any(url: str | None, width: int) -> str | None:
    if not url: 
        return None
    # Try Commons first
    cm = extract_commons_filename(url)
    if cm:
        return f"https://commons.wikimedia.org/wiki/Special:FilePath/{cm}?width={width}"
    # Otherwise assume it’s a direct http(s) image
    if isinstance(url, str) and url.lower().startswith(("http://","https://")):
        return url
    return None

def osm_map_link(lat: float, lon: float, zoom: int = 16) -> str:
    return f"https://www.openstreetmap.org/?mlat={lat:.6f}&mlon={lon:.6f}#map={zoom}/{lat:.6f}/{lon:.6f}"

# -------------------- 1) WIKIDATA SAT POINTS --------------------
print("🔎 Fetching SAT points from Wikidata…")
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(
    """
    SELECT ?item ?itemLabel ?coord ?image ?website WHERE {
      ?item wdt:P6104 wd:Q134294510;  # SAT point membership
            wdt:P625 ?coord.
      OPTIONAL { ?item wdt:P18 ?image. }
      OPTIONAL { ?item wdt:P856 ?website. }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
    }
    """
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

wd_points = []
for r in results["results"]["bindings"]:
    coord = r["coord"]["value"].replace("Point(", "").replace(")", "").split(" ")
    lon, lat = map(float, coord)
    item_uri = r["item"]["value"]
    qid = item_uri.rsplit("/", 1)[-1]

    # Image sources
    p18 = r.get("image", {}).get("value")
    # If you’re NOT selecting ?osmimage in SPARQL, this will always be None – that’s fine.
    osm_img = r.get("osmimage", {}).get("value")
    if not p18 and not osm_img:
        # Overpass fallback near the coordinate
        osm_img = find_osm_image_near(lat, lon)

    img_final = p18 or osm_img

    wd_points.append({
        "name": r["itemLabel"]["value"],
        "lat": lat,
        "lon": lon,
        "image": img_final,
        "image_thumb": thumb_from_any(img_final, CONFIG["thumb_width"]),
        "qid": qid,
        "website": r.get("website", {}).get("value"),
        "osm_map": osm_map_link(lat, lon, 16),  # OSM link fallback even without P10689
    })

# Build GeoDataFrame (you were missing this)
gdf_wd = gpd.GeoDataFrame(
    wd_points, geometry=[Point(p["lon"], p["lat"]) for p in wd_points], crs="EPSG:4326"
)


# -------------------- 2) OSM AMENITIES (TOILETS + WATER) --------------------
print("🚰 Fetching OSM amenities (toilets & drinking water)…")
south, west, north, east = CONFIG["bbox"]
query_osm = f"""
[out:json][timeout:60];
(
  node["amenity"="toilets"]({south},{west},{north},{east});
  node["amenity"="drinking_water"]({south},{west},{north},{east});
);
out center tags;
"""
osm_json = overpass(query_osm)

osm_points = []
for e in osm_json.get("elements", []):
    if e.get("type") != "node":
        continue
    tags = e.get("tags", {})
    amenity = tags.get("amenity")
    name = tags.get("name") or amenity
    osm_id = e.get("id")
    wheelchair = tags.get("wheelchair")  # yes/no/limited
    paper = tags.get("toilets:paper")  # yes/no
    operator_ = tags.get("operator")
    id_link = f"https://www.openstreetmap.org/edit?editor=id&node={osm_id}"
    osm_link = f"https://www.openstreetmap.org/node/{osm_id}"
    note_link = f"https://www.openstreetmap.org/note/new#map=18/{e.get('lat')}/{e.get('lon')}"

    osm_points.append({
        "name": name,
        "lat": e.get("lat"),
        "lon": e.get("lon"),
        "amenity": amenity,
        "wheelchair": wheelchair,
        "paper": paper,
        "operator": operator_,
        "osm_id": osm_id,
        "id_link": id_link,
        "osm_link": osm_link,
        "note_link": note_link,
    })

gdf_osm = gpd.GeoDataFrame(
    osm_points, geometry=[Point(p["lon"], p["lat"]) for p in osm_points], crs="EPSG:4326"
)

# -------------------- 3) SAT SECTIONS + OSM GEOMETRIES --------------------
print("🧭 Fetching SAT sections + OSM geometries…")
trail_qid = CONFIG["trail_qid"]
sparql_sections = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql_sections.setQuery(
    f"""
    SELECT ?section ?sectionLabel ?osmr ?osmw_p10689 ?osmw_p11693 ?p856 ?p973 WHERE {{
      VALUES ?trail {{ wd:{trail_qid} }}
      {{ ?section wdt:P361+ ?trail . }} UNION {{ ?trail wdt:P527 ?section . }}
      FILTER(?section != ?trail)
      OPTIONAL {{ ?section wdt:P402 ?osmr }}
      OPTIONAL {{ ?section wdt:P10689 ?osmw_p10689 }}
      OPTIONAL {{ ?section wdt:P11693 ?osmw_p11693 }}
      OPTIONAL {{ ?section wdt:P856 ?p856 }}
      OPTIONAL {{ ?section wdt:P973 ?p973 }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    ORDER BY ?sectionLabel
    """
)
sparql_sections.setReturnFormat(JSON)
sec_res = sparql_sections.query().convert()

sections = []
for b in sec_res["results"]["bindings"]:
    label = b.get("sectionLabel", {}).get("value")
    qid = b.get("section", {}).get("value", "").rsplit("/", 1)[-1]
    osmr = b.get("osmr", {}).get("value")  # relation id
    w_p10689 = b.get("osmw_p10689", {}).get("value")  # OSM way id (old prop)
    w_p11693 = b.get("osmw_p11693", {}).get("value")  # OSM way id (new prop)
    p856 = b.get("p856", {}).get("value")
    p973 = b.get("p973", {}).get("value")

    # prefer SSS link if either P973 or P856 contains that domain
    sss_url = None
    for candidate in [p973, p856]:
        if sss_like_url(candidate):
            sss_url = candidate
            break

    sections.append({
        "label": label,
        "qid": qid,
        "osmr": osmr,
        "ways": [w for w in [w_p10689, w_p11693] if w],
        "sss": sss_url or p973 or p856,
    })

# Fetch geometries from Overpass for each section
section_lines = []
for sec in sections:
    label = sec["label"]
    osmr = sec["osmr"]
    ways = sec["ways"]

    if osmr:
        # Prefer relation -> expand to member ways
        ov_q = f"""
        [out:json][timeout:180];
        relation({osmr});
        way(r);
        out tags geom;
        """
    elif ways:
        # Fallback: one or more specific ways
        parts_str = "\n".join([f"way({w});" for w in ways])

        ov_q = f"""
        [out:json][timeout:180];
        (
        {parts_str}
        );
        out tags geom;
        """
    else:
        continue

    data = overpass(ov_q)
    if not data.get("elements"):
        continue
    for el in data.get("elements", []):
        if el.get("type") == "way" and "geometry" in el:
            coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
            if len(coords) >= 2:
                line = LineString(coords)
                section_lines.append({"geometry": line, "label": label, "qid": sec["qid"], "link": sec["sss"]})

if section_lines:
    gdf_sections = gpd.GeoDataFrame(section_lines, geometry=[s["geometry"] for s in section_lines], crs="EPSG:4326")
else:
    gdf_sections = gpd.GeoDataFrame(columns=["geometry","label","qid","link"], geometry=[], crs="EPSG:4326")

# -------------------- 4) EXPORT VERSIONED GEOJSON --------------------
ensure_dir(CONFIG["out_dir"])
print("💾 Writing versioned GeoJSON…")
gdf_wd.to_file(CONFIG["out_points"], driver="GeoJSON")
gdf_osm.to_file(CONFIG["out_facilities"], driver="GeoJSON")
if not gdf_sections.empty:
    gdf_sections.to_file(CONFIG["out_sections"], driver="GeoJSON")

# -------------------- 5) BUILD MAP --------------------
print("🗺️ Building fancy map (v2.3)…")
m = folium.Map(location=CONFIG["map_center"], zoom_start=8, control_scale=True, tiles="OpenStreetMap")

# Feature groups
fg_wd = folium.FeatureGroup(name="SAT-punkter (Wikidata)", show=True)
# Replace single OSM group with sub-groups for better filtering
fg_toilets_wc = folium.FeatureGroup(name="Toalett (rullstol)", show=True)
fg_toilets_paper = folium.FeatureGroup(name="Toalett (papper)", show=False)
fg_toilets_other = folium.FeatureGroup(name="Toalett (övriga)", show=True)
fg_water = folium.FeatureGroup(name="Dricksvatten", show=True)
fg_sections = folium.FeatureGroup(name="SAT-sektioner", show=True)

# Clusters for each group
cl_wc = MarkerCluster(name="WC rullstol").add_to(fg_toilets_wc)
cl_paper = MarkerCluster(name="WC med papper").add_to(fg_toilets_paper)
cl_toilets = MarkerCluster(name="WC övriga").add_to(fg_toilets_other)
cl_water = MarkerCluster(name="Dricksvatten kluster").add_to(fg_water)
wd_cluster = MarkerCluster(name="SAT-punkter kluster").add_to(fg_wd)

# Icon setup (wheelchair-friendly gets a distinct icon/color)
ICON_TOILET = {"color": "red", "icon": "male", "prefix": "fa", "label": "Toalett"}
ICON_TOILET_WC = {"color": "darkgreen", "icon": "wheelchair", "prefix": "fa", "label": "Toalett (rullstol)"}
ICON_TOILET_PAPER = {"color": "orange", "icon": "sticky-note", "prefix": "fa", "label": "Toalett (papper)"}
ICON_WATER = {"color": "blue", "icon": "tint", "prefix": "fa", "label": "Dricksvatten"}

# Add WD markers with image popups (<img src="…"/>)
for _, row in gdf_wd.iterrows():
    title = html.escape(str(row.get("name", "(namnlös)")))
    qid = row.get("qid")
    wikidata_link = f"https://www.wikidata.org/wiki/{qid}" if qid else None
    website = row.get("website")
    img = row.get("image_thumb")
    osm_link_at_pos = row.get("osm_map")

    parts = [f"<h4 style='margin:0 0 6px 0;'>{title}</h4>"]
    if img:
        parts.append(
            f"<img src='{html.escape(str(img))}' alt='{title}' "
            f"style='max-width:360px;display:block;margin-bottom:6px;' loading='lazy'>"
        )
    else:
        parts.append("<div style='color:#888'>Ingen bild (P18/OSM)</div>")

    links = []
    if wikidata_link:
        links.append(f"<a href='{wikidata_link}' target='_blank' rel='noopener'>Wikidata</a>")
    if website:
        links.append(f"<a href='{html.escape(str(website))}' target='_blank' rel='noopener'>Webbplats</a>")
    if osm_link_at_pos:
        links.append(f"<a href='{html.escape(str(osm_link_at_pos))}' target='_blank' rel='noopener'>OSM</a>")

    if links:
        parts.append("<div>" + " | ".join(links) + "</div>")

    popup_html = "".join(parts)

    folium.Marker(
        [row["lat"], row["lon"]],
        popup=folium.Popup(popup_html, max_width=400),
        tooltip=title,
        icon=folium.Icon(color="green", icon="info-sign"),
    ).add_to(wd_cluster)


# Add OSM markers into specific groups
for _, row in gdf_osm.iterrows():
    amenity = row.get("amenity")
    if amenity == "drinking_water":
        cfg = ICON_WATER
        cluster_target = cl_water
        tooltip = cfg["label"]
    elif amenity == "toilets":
        wc = (row.get("wheelchair") or "").lower() == "yes"
        has_paper = (row.get("paper") or "").lower() == "yes"
        if wc:
            cfg = ICON_TOILET_WC
            cluster_target = cl_wc
        elif has_paper:
            cfg = ICON_TOILET_PAPER
            cluster_target = cl_paper
        else:
            cfg = ICON_TOILET
            cluster_target = cl_toilets
        tooltip = cfg["label"]
    else:
        continue

    extras = []
    if amenity == "toilets":
        wc_val = row.get("wheelchair")
        paper_val = row.get("paper")
        operator_ = row.get("operator")
        if wc_val:
            extras.append(f"Rullstol: <strong>{html.escape(wc_val)}</strong>")
        if paper_val:
            extras.append(f"Papper: <strong>{html.escape(paper_val)}</strong>")
        if operator_:
            extras.append(f"Operatör: <strong>{html.escape(operator_)}</strong>")

    quick = (
        f"<a href='{row['osm_link']}' target='_blank' rel='noopener'>Visa i OSM</a> · "
        f"<a href='{row['id_link']}' target='_blank' rel='noopener'>Redigera</a> · "
        f"<a href='{row['note_link']}' target='_blank' rel='noopener'>Rapportera status</a>"
    )

    popup_html = (
        f"<h5 style='margin:0 0 6px 0;'>{html.escape(str(row.get('name', tooltip)))}</h5>"
        f"<div style='margin-bottom:6px;color:#444'>{html.escape(tooltip)}</div>"
        + ("<div>" + "<br>".join(extras) + "</div>" if extras else "")
        + f"<div style='margin-top:6px'>{quick}</div>"
    )

    folium.Marker(
        [row["lat"], row["lon"]],
        popup=folium.Popup(popup_html, max_width=360),
        tooltip=tooltip,
        icon=folium.Icon(color=cfg["color"], icon=cfg["icon"], prefix=cfg.get("prefix", "fa")),
    ).add_to(cluster_target)

# Add section polylines (dark red) with mouseover name and links
if not gdf_sections.empty:
    for _, s in gdf_sections.iterrows():
        name = s.get("label") or "SAT-sektion"
        tooltip = f"SAT {html.escape(name)}"
        link = s.get("link") or (f"https://www.wikidata.org/wiki/{s.get('qid')}")
        popup = f"<strong>{tooltip}</strong><br><a href='{html.escape(link)}' target='_blank' rel='noopener'>Mer info</a>"
        folium.PolyLine(
            locations=[(lat, lon) for lon, lat in zip(*s.geometry.xy)],
            color=CONFIG["trail_color"],
            weight=CONFIG["trail_weight"],
            opacity=0.9,
            tooltip=tooltip,
            popup=folium.Popup(popup, max_width=320),
        ).add_to(fg_sections)

# Add groups
fg_wd.add_to(m)
fg_toilets_wc.add_to(m)
fg_toilets_paper.add_to(m)
fg_toilets_other.add_to(m)
fg_water.add_to(m)
fg_sections.add_to(m)

# Add Legend
legend_html = f'''
<div style="position: fixed; bottom: 20px; left: 20px; z-index: 9999; background: white; padding: 10px 12px; border: 1px solid #ccc; border-radius: 8px; box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px; line-height: 1.35;">
  <div style="font-weight:600; margin-bottom:6px;">Teckenförklaring</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:{CONFIG['trail_color']};margin-right:6px;"></span> SAT-sektion (mörkröd)</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:#dc3545;margin-right:6px;"></span> Toalett</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:#198754;margin-right:6px;"></span> Toalett (rullstol)</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:#fd7e14;margin-right:6px;"></span> Toalett (papper)</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:#0d6efd;margin-right:6px;"></span> Dricksvatten</div>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# Controls
folium.LayerControl(collapsed=False).add_to(m)

# Save map (v2.3)
ensure_dir(CONFIG["out_dir"])
m.save(CONFIG["out_html"])
print("✅ Export klar:", CONFIG["out_points"], CONFIG["out_facilities"], CONFIG["out_sections"], CONFIG["out_html"])


🔎 Fetching SAT points from Wikidata…
🚰 Fetching OSM amenities (toilets & drinking water)…
🧭 Fetching SAT sections + OSM geometries…


In [ ]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
